In [1]:
# Импорт необходимых библиотек
import cv2
import numpy as np
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
from skimage.color import rgb2lab

In [2]:
# Функция для сегментации методом k-средних
def segment_kmeans(image, color_space='RGB', num_clusters=3):
    if color_space == 'Lab':
        image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    pixel_values = image.reshape((-1, 3))
    pixel_values = np.float32(pixel_values)
    kmeans = KMeans(n_clusters=num_clusters, random_state=0)
    labels = kmeans.fit_predict(pixel_values)
    centers = np.uint8(kmeans.cluster_centers_)
    segmented_image = centers[labels.flatten()]
    segmented_image = segmented_image.reshape(image.shape)
    return segmented_image

# Функция для сегментации методом DBSCAN
def segment_dbscan(image, color_space='RGB', eps=10, min_samples=50):
    if color_space == 'Lab':
        image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    pixel_values = image.reshape((-1, 3))
    pixel_values = np.float32(pixel_values)
    pixel_values = StandardScaler().fit_transform(pixel_values)
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(pixel_values)
    labels_unique = np.unique(labels)
    segmented_image = np.zeros((image.shape), dtype=np.uint8)
    for label in labels_unique:
        mask = labels.reshape(image.shape[:-1]) == label
        segmented_image[mask] = np.median(image[mask], axis=0)
    return segmented_image

# Функция для сегментации путем выращивания семян
def segment_seeds(image, num_seeds=5, intensity_difference=30):
    seeds = np.zeros(image.shape[:2], np.int32)
    for i in range(num_seeds):
        x = np.random.randint(image.shape[1])
        y = np.random.randint(image.shape[0])
        seeds[y, x] = i+1
    connectivity = 8
    flags = connectivity | cv2.FLOODFILL_FIXED_RANGE | cv2.FLOODFILL_MASK_ONLY
    mask = np.zeros((image.shape[0] + 2, image.shape[1] + 2), np.uint8)
    for y in range(image.shape[0]):
        for x in range(image.shape[1]):
            if seeds[y, x]:
                cv2.floodFill(image, mask, (x, y), None, (intensity_difference,)*3, (intensity_difference,)*3, flags)
    return mask[1:-1, 1:-1]

# Функция для сегментации методом разрастания областей
def segment_region_growing(image, seed=None, intensity_threshold=10):
    if seed is None:
        seed = (0, 0)
    segmented = np.zeros_like(image)
    segmented[seed] = 255
    current_points = [seed]
    while current_points:
        new_points = []
        for point in current_points:
            neighbors = [((point[0] + dx), (point[1] + dy)) for dx in range(-1, 2) for dy in range(-1, 2) if dx != 0 or dy != 0]
            for nx, ny in neighbors:
                if 0 <= nx < image.shape[0] and 0 <= ny < image.shape[1]:
                    if segmented[nx, ny] == 0 and abs(int(image[nx, ny]) - int(image[point])) < intensity_threshold:
                        segmented[nx, ny] = 255
                        new_points.append((nx, ny))
        current_points = new_points
    return segmented

# Функция для сегментации методом водораздела
def segment_watershed(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    kernel = np.ones((3,3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
    sure_bg = cv2.dilate(opening, kernel, iterations=3)
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    ret, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)
    ret, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[unknown==255] = 0
    markers = cv2.watershed(image, markers)
    image[markers == -1] = [255, 0, 0]
    return image

# Функция для обработки видео
def process_video(video_path, output_path, segmentation_function):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = None
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        segmented_frame = segmentation_function(frame)
        if out is None:
            out = cv2.VideoWriter(output_path, fourcc, 20.0, (segmented_frame.shape[1], segmented_frame.shape[0]))
        out.write(segmented_frame)
    cap.release()
    out.release()
    cv2.destroyAllWindows()
# Пример использования
# process_video('path_to_input_video.avi', 'path_to_output_video.avi', segment_watershed)

In [3]:
# import cv2
# import matplotlib.pyplot as plt

def load_and_show_segmentations(image_path):
    # Загрузка изображения
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Применение методов сегментации
    segmented_kmeans_rgb = segment_kmeans(image, 'RGB', 3)
    segmented_kmeans_lab = segment_kmeans(image, 'Lab', 3)
    segmented_dbscan_rgb = segment_dbscan(image, 'RGB', eps=0.5, min_samples=50)
    segmented_dbscan_lab = segment_dbscan(image, 'Lab', eps=0.5, min_samples=50)
    segmented_seeds = segment_seeds(image.copy(), num_seeds=10, intensity_difference=30)
    segmented_region_growing = segment_region_growing(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
    segmented_watershed = segment_watershed(image.copy())

    # Отображение результатов
    fig, ax = plt.subplots(3, 3, figsize=(15, 10))
    ax[0, 0].imshow(image_rgb)
    ax[0, 0].set_title('Original Image')
    ax[0, 0].axis('off')

    ax[0, 1].imshow(segmented_kmeans_rgb)
    ax[0, 1].set_title('K-Means RGB')
    ax[0, 1].axis('off')

    ax[0, 2].imshow(segmented_kmeans_lab)
    ax[0, 2].set_title('K-Means Lab')
    ax[0, 2].axis('off')

    ax[1, 0].imshow(segmented_dbscan_rgb)
    ax[1, 0].set_title('DBSCAN RGB')
    ax[1, 0].axis('off')

    ax[1, 1].imshow(segmented_dbscan_lab)
    ax[1, 1].set_title('DBSCAN Lab')
    ax[1, 1].axis('off')

    ax[1, 2].imshow(segmented_seeds)
    ax[1, 2].set_title('Seed Segmentation')
    ax[1, 2].axis('off')

    ax[2, 0].imshow(segmented_region_growing, cmap='gray')
    ax[2, 0].set_title('Region Growing')
    ax[2, 0].axis('off')

    ax[2, 1].imshow(segmented_watershed)
    ax[2, 1].set_title('Watershed')
    ax[2, 1].axis('off')

    plt.tight_layout()
    plt.show()

In [4]:
import cv2
import matplotlib.pyplot as plt

In [5]:
image = cv2.imread('../assets/1000_F_532804846_aW0Agk6JM4KOiz4cZFTw7ALx7z8bsOUQ.jpg')
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [6]:
segmented_kmeans_lab = segment_kmeans(image, 'Lab', 3)

In [7]:
segmented_kmeans_rgb = segment_kmeans(image, 'RGB', 3)

In [ ]:
segmented_dbscan_rgb = segment_dbscan(image, 'RGB', eps=0.5, min_samples=50)

In [ ]:
segmented_dbscan_lab = segment_dbscan(image, 'Lab', eps=0.5, min_samples=50)

In [ ]:
segmented_seeds = segment_seeds(image.copy(), num_seeds=10, intensity_difference=30)

In [ ]:
segmented_region_growing = segment_region_growing(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
segmented_watershed = segment_watershed(image.copy())